## load libs

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from process_images import *
from pystripe.core import *
import matplotlib.pyplot as plt
def plot_images(img_list: List[ndarray], img_labels: List[str], vmax: int):
    if len(img_list) == 1:
        plt.figure(figsize=(20, 20))
        plt.imshow(img_list[0], cmap='gray', vmin=0, vmax=vmax)
        plt.title(img_labels[0])
    else:
        fig, axes = plt.subplots(nrows=1, ncols=len(img_list), figsize=(20, 20))
        for idx, (im, label) in enumerate(zip(img_list, img_labels)):
            axes[idx].imshow(im, cmap='gray', vmin=0, vmax=vmax)
            axes[idx].set_title(label)
    plt.tight_layout()
    plt.show()

In [ ]:
from parallel_image_processor import *
tsv_volume = TSVVolume.load(r'E:\20230510_13_34_13_SM230308_05_LS_15x_800z_MIP_stitched\Ex_488_Em_525_MIP_xml_import_step_5.xml')
shape: Tuple[int, int, int] = tsv_volume.volume.shape  # shape is in z y x format
img = tsv_volume.imread(
    VExtent(
        tsv_volume.volume.x0, tsv_volume.volume.x1,
        tsv_volume.volume.y0, tsv_volume.volume.y1,
        tsv_volume.volume.z0 + shape[0]//2, tsv_volume.volume.z0 + shape[0]//2 + 1),
    tsv_volume.dtype)[0]
parallel_image_processor(
    source=TSVVolume.load(r'/data/20230419_17_34_03_SM221011_06_LS_15x_800z_stitched/Ex_488_Em_525_xml_import_step_5.xml'),
    destination=r"/data/20230419_17_34_03_SM221011_06_LS_15x_800z_stitched/Ex_488_Em_525_tif",
    fun=process_img,
    kwargs={'bleach_correction_frequency': 0.0005, 'bleach_correction_max_method': False, 'bleach_correction_y_slice_max': None, 'threshold': None, 'sigma': (4000.0, 4000.0), 'bidirectional': True, 'lightsheet': False, 'percentile': 0.25, 'rotate': 90, 'convert_to_8bit': False, 'bit_shift_to_right': 8, 'tile_size': (39220, 28056), 'd_type': 'uint16', "verbose": True},
    source_voxel=(0.8, 0.4, 0.4),
    target_voxel=20,
    max_processors=1
)

In [ ]:
def get_layer(
    index: int,          # layer of image requested
    image: ndarray,      # 3-D image (use TifStack.as_3d_numpy())
    plane = "xy",        # must be "xy", "yx", "xz", "zx", "yz", "zy"
    img_format = "zyx",  # xyz in some order
):
    # guards
    if plane not in {"xy", "yx", "xz", "zx", "yz", "zy"} or img_format not in {"zyx", "zxy", "yxz", "yzx", "xyz", "xzy"}:
        print(f"Invalid plane selected in get_layer().  Plane: {plane}, Layer: {index}, Img_format: {img_format}\nReturning to caller...")
        return None

    # get the layer
    if 'x' not in plane:   sub = img_format.index('x')
    elif 'y' not in plane: sub = img_format.index('y')
    elif 'z' not in plane: sub = img_format.index('z')

    if sub == 0:   layer_image = image[index, :, :]
    elif sub == 1: layer_image = image[:, index, :]
    elif sub == 2: layer_image = image[:, :, index]

    # if plane is flipped compared to image format, return the transpose.
    if plane not in (img_format[:sub] + img_format[sub + 1:]):
        return layer_image.transpose()
    return layer_image

# run to rotate images
from os import makedirs

######## CHANGE THESE ########
cha1_path = "D:/BMAP/Brain 4/cha1"
cha2_path = "D:/BMAP/Brain 4/cha2"
cha3_path = "D:/BMAP/Brain 4/cha3"
# set to None if nothing to convert

##############################
if cha1_path:
    makedirs(cha1_path + "_zx", exist_ok=True)
    makedirs(cha1_path + "_zy", exist_ok=True)
    stack1 = TifStack(cha1_path).as_3d_numpy()
    for i in range(stack1.shape[1]):
        imwrite(cha1_path + "_zx/" + str(i + 1) + ".tif", get_layer(i, stack1, "zx"))
    for i in range(stack1.shape[2]):
        imwrite(cha1_path + "_zy/" + str(i + 1) + ".tif", get_layer(i, stack1, "zy"))
if cha2_path:
    makedirs(cha2_path + "_zx", exist_ok=True)
    makedirs(cha2_path + "_zy", exist_ok=True)
    stack2 = TifStack(cha2_path).as_3d_numpy()
    for i in range(stack2.shape[1]):
        imwrite(cha2_path + "_zx/" + str(i + 1) + ".tif", get_layer(i, stack2, "zx"))
    for i in range(stack2.shape[2]):
        imwrite(cha2_path + "_zy/" + str(i + 1) + ".tif", get_layer(i, stack2, "zy"))
if cha3_path:
    makedirs(cha3_path + "_zx", exist_ok=True)
    makedirs(cha3_path + "_zy", exist_ok=True)
    stack3 = TifStack(cha3_path).as_3d_numpy()
    for i in range(stack3.shape[1]):
        imwrite(cha3_path + "_zx/" + str(i + 1) + ".tif", get_layer(i, stack3, "zx"))
    for i in range(stack3.shape[2]):
        imwrite(cha3_path + "_zy/" + str(i + 1) + ".tif", get_layer(i, stack3, "zy"))
print("Operation Completed")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_data(file_path:str, title:str, label1:str, label2:str):
    with open(file_path, "r") as file:
        line = file.readline()

        x_offset = [0]
        y_offset = [0]

        while line:
            if "Skipped" in line:
                x_offset.append(0)
                y_offset.append(0)
                line = file.readline()
                continue
            s = line.split(",")
            x_offset.append(float(s[-2][:-1]))
            line = file.readline()
            s = line.split(",")
            y_offset.append(float(s[-2][:-1]))
            line = file.readline()
            # skip last line
            line = file.readline()

        plt.plot(range(len(x_offset)), x_offset, label=label1)
        plt.plot(range(len(y_offset)), y_offset, label=label2)
        plt.title(title)
        plt.xlabel("Layer index")
        plt.ylabel("Units")
        plt.ylim((-20, 20))
        plt.legend()
        plt.show()

plot_data("D:/BMAP/Brain 4/offsets/zy_matrices_im12.txt", "Image alignment offsets of Images 1 and 2 from zy-slices", "z-offset", "y-offset")
plot_data("D:/BMAP/Brain 4/offsets/zy_matrices_im13.txt", "Image alignment offsets of Images 1 and 3 from zy-slices", "z-offset", "y-offset")
plot_data("D:/BMAP/Brain 4/offsets/zx_matrices_im12.txt", "Image alignment offsets of Images 1 and 2 from zx-slices", "z-offset", "x-offset")
plot_data("D:/BMAP/Brain 4/offsets/zx_matrices_im13.txt", "Image alignment offsets of Images 1 and 3 from zx-slices", "z-offset", "x-offset")
plot_data("D:/BMAP/Brain 4/offsets/xy_matrices_im12.txt", "Image alignment offsets of Images 1 and 2 from xy-slices", "x-offset", "y-offset")
plot_data("D:/BMAP/Brain 4/offsets/xy_matrices_im13.txt", "Image alignment offsets of Images 1 and 3 from xy-slices", "x-offset", "y-offset")

In [1]:
from align_images import align_all_images, resize_arrays
from supplements.tifstack import TifStack

cha1 = TifStack("C:/Users/adnja/Documents/GitHub/image-preprocessing-pipeline/tempData/cha1").as_3d_numpy()
cha2 = TifStack("C:/Users/adnja/Documents/GitHub/image-preprocessing-pipeline/tempData/cha2").as_3d_numpy()
cha3 = TifStack("C:/Users/adnja/Documents/GitHub/image-preprocessing-pipeline/tempData/cha3").as_3d_numpy()
max_iterations = 50

channels = resize_arrays([cha1, cha2, cha3])
print("test")
for i in channels:
    print(i.shape)
alignments, residuals = align_all_images(channels, verbose=True)

print("Alignment: ", end='')
print(alignments)
print("Residuals: ", end='')
print(residuals)

(791, 1132, 1272)
test
(791, 1132, 1272)
(791, 1132, 1272)
(791, 1132, 1272)
(791, 1132, 1272)
Loaded images.
Resized shapes: (791, 1132, 1272)
Iteration 0
downsampling factor for transformation_matrix 1
[[ 1.    0.   -0.56]
 [ 0.    1.    1.01]
 [ 0.    0.    1.  ]]
downsampling factor for transformation_matrix 1
[[ 1.    0.   -0.08]
 [ 0.    1.    0.41]
 [ 0.    0.    1.  ]]
downsampling factor for transformation_matrix 1
[[ 1.    0.    2.07]
 [ 0.    1.   -0.39]
 [ 0.    0.    1.  ]]
0
0
1
Iteration 1
downsampling factor for transformation_matrix 1
[[ 1.    0.   -0.3 ]
 [ 0.    1.    0.84]
 [ 0.    0.    1.  ]]
downsampling factor for transformation_matrix 1
[[ 1.    0.   -0.98]
 [ 0.    1.    0.43]
 [ 0.    0.    1.  ]]
downsampling factor for transformation_matrix 1
[[ 1.    0.    1.09]
 [ 0.    1.   -0.41]
 [ 0.    0.    1.  ]]
0
0
0
Images converged.
Residual: (-0.637891411781311, 0.21902790665626526, 0.7578848600387573)
(791, 1132, 1272)
Loaded images.
Resized shapes: (791, 113


* make align_images make copy
* function that takes in array of stacks and returns array of offsets
    * choose reference image

* submit refactored code

* try using mask for better edge detection
* Sobel operator -> mask -> align